Apply the most compatible ML algorithm and build a text classification model to predict "Article Types"

Let's divide the classification problem into the following steps:

1. Preprocessing and cleaning

2. For vectorization, use SentenceBERT or other BERT tokenizer.

3. Select and train an ML classifier model

4. Hyperparameter tuning

5. Validate and evaluate the Accuracy (Use Cross-validation methods)

6. Present the performance metrics

7. Save and Reload the model from the disk

8. Extract the heading and Full article from the article_url unknown articles.csv

9. Predict the "Article_Types" for the extracted data

# Data Cleaning and Pre-processing

In [5]:
import pandas as pd
import numpy as np

In [6]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [7]:
# Load the dataset
df = pd.read_csv('articles.csv', encoding='ISO-8859-1')

In [8]:
df.shape

(4305, 8)

In [9]:
df.head()

,Id,Heading,Article.Banner.Image,Outlets,Article.Description,Full_Article,Article_Type,Tonality
0,d6995462-5e87-453b-b64d-e9f1df6e94d2,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",NaN,Essex Caller,<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,8b05e939-a89e-4548-b92b-013822e8ee7d,Bells Nexus Air Taxi Concept Rings Changes Fo...,NaN,Aviation Week Network,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,69fcd400-bceb-4255-8277-619f2d68ac0b,Bell Helicopter Show Air Taxi Nexus,http://images.tmtpost.com/uploads/images/2019/...,TMTPost,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,17943578-c11b-414b-b3f5-063d3a93157b,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,http://www.fredzone.org/wp-content/uploads/201...,Fredzone,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,f33c7b11-5f77-4a98-bb2e-d36689042aea,Les premiers retours dOlivier Ezratty,NaN,FrenchWeb,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive


In [10]:
# Check for null values and data type of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4305 entries, 0 to 4304
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Id                    4305 non-null   object
 1   Heading               4305 non-null   object
 2   Article.Banner.Image  1753 non-null   object
 3   Outlets               4305 non-null   object
 4   Article.Description   4305 non-null   object
 5   Full_Article          4305 non-null   object
 6   Article_Type          4305 non-null   object
 7   Tonality              3873 non-null   object
dtypes: object(8)
memory usage: 269.2+ KB


In [11]:
len(df['Outlets'].unique())

1762

In [12]:
# Drop irrelevant columns
df1 = df.drop(['Id', 'Article.Banner.Image', 'Outlets'], axis=1)
df1.shape

(4305, 5)

In [13]:
# Check for null values
df1.isnull().sum()

Heading                  0
Article.Description      0
Full_Article             0
Article_Type             0
Tonality               432
dtype: int64

In [14]:
# Fill null values
df1['Tonality'] = df1['Tonality'].fillna(df1['Tonality'].mode()[0])

In [15]:
df1.isnull().sum()

Heading                0
Article.Description    0
Full_Article           0
Article_Type           0
Tonality               0
dtype: int64

In [16]:
df1['Article_Type'].unique()

array(['Commercial', 'Military', 'Training', 'Executives', 'Others',
       'Financing', 'Support & Services'], dtype=object)

In [17]:
df1['Tonality'].unique()

array(['Negative', 'Positive', 'Neutral'], dtype=object)

In [18]:
df2 = df1.copy()
df2.head()

,Heading,Article.Description,Full_Article,Article_Type,Tonality
0,"A Puzzling Maneuver, Then Freefall: NTSB Repor...",<p>The helicopter that crashed in Southeast Al...,<p>The helicopter that crashed in Southeast Al...,Commercial,Negative
1,Bells Nexus Air Taxi Concept Rings Changes Fo...,<p>A year after teasing the fledgling electric...,<p>A year after teasing the fledgling electric...,Commercial,Positive
2,Bell Helicopter Show Air Taxi Nexus,<p>Bell released the full-size design of the v...,<p>Bell released the full-size design of the v...,Commercial,Positive
3,BELL DÉVOILE LA CONCEPTION INTÉGRALE DE SON TA...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,<p>Bell est une soci&eacute;t&eacute; am&eacut...,Commercial,Positive
4,Les premiers retours dOlivier Ezratty,<p>It was still anecdotal to observe the explo...,<p>It was still anecdotal to observe the explo...,Commercial,Positive


In [19]:
# Encode categorical columns using Label Encoder

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df2['Tonality'] = le.fit_transform(df2['Tonality'])
df2['Article_Type'] = le.fit_transform(df2['Article_Type'])

In [20]:
df2['Article_Type'].unique()

array([0, 3, 6, 1, 4, 2, 5])

In [21]:
df2['Tonality'].unique()

array([0, 2, 1])

# Vectorization

In [22]:
# Install the Sentence Transformers library, which is a Python module for SentenceBERT developed by the UKPLab
!pip3 install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3a764ca85c91cebca175b50439c022751ab02b776d77b85d1abb41d715342efb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [23]:
# Load a pre-trained SentenceBERT model. The 'bert-base-nli-mean-tokens' model is commonly used

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

for column in ['Heading', 'Article.Description', 'Full_Article']:
    df2[column + '_embedding'] = df2[column].apply(lambda x: model.encode(x))

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [24]:
df3 = df2.drop(['Heading', 'Article.Description', 'Full_Article'], axis=1)
df3.head()

,Article_Type,Tonality,Heading_embedding,Article.Description_embedding,Full_Article_embedding
0,0,0,"[0.0068183946, 0.38549194, 0.11103279, 0.25931...","[-0.30879164, 0.47476262, 0.17605324, 0.219102...","[-0.2975319, 0.51373106, 0.20075294, 0.2210167..."
1,0,2,"[0.13352083, -0.045963082, 2.3014436, 0.146599...","[-0.22241966, 0.1363592, 1.0301527, 0.088684, ...","[-0.22241966, 0.1363592, 1.0301527, 0.088684, ..."
2,0,2,"[0.13152, -0.07194051, 1.0037086, 0.19439247, ...","[-0.6536699, -0.106779926, 1.3116853, 0.115212...","[-0.6536699, -0.106779926, 1.3116853, 0.115212..."
3,0,2,"[-0.31541976, 0.7637212, 1.5530186, 0.58139193...","[-0.35611284, 0.0979317, 1.6341908, 0.17869571...","[-0.35611284, 0.0979317, 1.6341908, 0.17869571..."
4,0,2,"[-0.2785986, 0.16457765, 0.9981047, 0.41436026...","[-0.0018761967, 0.6525792, -0.44657305, 0.0285...","[-0.0018761967, 0.6525792, -0.44657305, 0.0285..."


In [25]:
df4 = df3[['Heading_embedding', 'Full_Article_embedding', 'Article_Type']]
df4.head()

,Heading_embedding,Full_Article_embedding,Article_Type
0,"[0.0068183946, 0.38549194, 0.11103279, 0.25931...","[-0.2975319, 0.51373106, 0.20075294, 0.2210167...",0
1,"[0.13352083, -0.045963082, 2.3014436, 0.146599...","[-0.22241966, 0.1363592, 1.0301527, 0.088684, ...",0
2,"[0.13152, -0.07194051, 1.0037086, 0.19439247, ...","[-0.6536699, -0.106779926, 1.3116853, 0.115212...",0
3,"[-0.31541976, 0.7637212, 1.5530186, 0.58139193...","[-0.35611284, 0.0979317, 1.6341908, 0.17869571...",0
4,"[-0.2785986, 0.16457765, 0.9981047, 0.41436026...","[-0.0018761967, 0.6525792, -0.44657305, 0.0285...",0


In [26]:
heading_features = pd.DataFrame(df4['Heading_embedding'].to_list(), columns=[f'heading_{i}' for i in range(len(df4['Heading_embedding'].iloc[0]))])
article_features = pd.DataFrame(df4['Full_Article_embedding'].to_list(), columns=[f'article_{i}' for i in range(len(df4['Full_Article_embedding'].iloc[0]))])

df5 = pd.concat([df4, heading_features, article_features], axis=1)

df5 = df5.drop(['Heading_embedding', 'Full_Article_embedding'], axis=1)

In [27]:
df5.head()

,Article_Type,heading_0,heading_1,heading_2,heading_3,heading_4,heading_5,heading_6,heading_7,heading_8,...,article_758,article_759,article_760,article_761,article_762,article_763,article_764,article_765,article_766,article_767
0,0,0.006818,0.385492,0.111033,0.259312,0.044005,-0.569938,0.613438,-0.476182,0.505468,...,-0.442507,0.213695,-1.048564,-0.371616,0.236706,-0.698086,-0.351208,-0.291637,0.164042,0.166120
1,0,0.133521,-0.045963,2.301444,0.146599,0.312994,-0.354764,-0.204951,-0.314788,0.620836,...,-0.498317,-1.014818,-1.643830,-0.490700,-0.124704,-0.598111,-0.384294,-0.424275,0.309889,-0.254794
2,0,0.131520,-0.071941,1.003709,0.194392,0.171645,0.060971,-0.475287,-0.187275,0.128736,...,-0.221055,-1.092432,-0.661865,-0.158220,-0.150076,-0.647646,-0.896942,-0.415517,-0.015467,-0.050569
3,0,-0.315420,0.763721,1.553019,0.581392,0.836126,-0.659893,-0.191330,0.306782,0.704866,...,0.129381,-0.636668,-0.731161,0.076023,0.134098,-1.099116,-0.230634,-0.127485,-0.038905,0.348545
4,0,-0.278599,0.164578,0.998105,0.414360,0.795069,-0.513556,-0.007940,0.652898,-0.087312,...,-0.360876,-0.186242,-1.036903,-0.809416,-0.027214,-1.038251,0.262545,0.132081,0.640065,-0.072544


In [28]:
x = df5.drop('Article_Type', axis=1)
y = df5['Article_Type']

# Training ML Classifier model

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [30]:
# Scale the input features using StandardScaler

from sklearn.preprocessing import MinMaxScaler

# Initialize StandardScaler
scaler = MinMaxScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(x_train_scaled, y_train)

LogisticRegression()

In [32]:
logistic_model.score(x_test_scaled, y_test)

0.908246225319396

### SVM

In [33]:
from sklearn.svm import SVC

svm_model = SVC()
svm_model.fit(x_train_scaled, y_train)

SVC()

In [34]:
svm_model.score(x_test_scaled, y_test)

0.9175377468060395

### Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier()
forest_model.fit(x_train_scaled, y_train)

RandomForestClassifier()

In [36]:
forest_model.score(x_test_scaled, y_test)

0.8815331010452961

### Multinomial Naive Bayes

In [37]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(x_train_scaled, y_train)

MultinomialNB()

In [38]:
naive_bayes_model.score(x_test_scaled, y_test)

0.818815331010453

# Hyperparameter Tuning and Cross-Validation

In [39]:
from sklearn.model_selection import GridSearchCV, cross_val_score

models = {
    'LogisticRegression': {
        'model': LogisticRegression(C=1),
        'params': {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
    },
    'SVM': {
        'model': SVC(C=1, gamma=1, kernel='rbf'),
        'params': {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']}
    },
    'MultinomialNB': {
    'model': MultinomialNB(),
    'params': {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
}
}

# Assuming you have your data in X (features) and y (labels)

for model_name, model_params in models.items():
    model = model_params['model']
    param_grid = model_params['params']

    grid_search = GridSearchCV(model, param_grid, cv=5)
    grid_search.fit(x_train_scaled, y_train)

    best_model = grid_search.best_estimator_

    # Cross-validation
    cross_val_scores = cross_val_score(best_model, x_train_scaled, y_train, cv=5)
    mean_cross_val_score = np.mean(cross_val_scores)

    # Evaluate the best model on the test set
    test_accuracy = best_model.score(x_test_scaled, y_test)

    print(f'{model_name} - Best Parameters: {grid_search.best_params_}')
    print(f'Mean Cross-Validation Score: {mean_cross_val_score}')
    print(f'Test Set Accuracy: {test_accuracy}\n')

LogisticRegression - Best Parameters: {'C': 1}
Mean Cross-Validation Score: 0.891985351199919
Test Set Accuracy: 0.908246225319396

SVM - Best Parameters: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
Mean Cross-Validation Score: 0.9085340230195429
Test Set Accuracy: 0.9279907084785134

MultinomialNB - Best Parameters: {'alpha': 0.001}
Mean Cross-Validation Score: 0.8138838728187128
Test Set Accuracy: 0.8199767711962834



In [41]:
# Choosing the best model and best parameters

best_model = SVC(C=100, gamma=0.01, kernel='rbf')
best_model.fit(x_train_scaled, y_train)

SVC(C=100, gamma=0.01)

In [43]:
y_pred = best_model.predict(x_test_scaled)

# Performance Metrics

### Accuracy

*   The ratio of correctly predicted instances to the total instances.
*   Suitable for balanced datasets.



In [44]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9279907084785134

### Precision

*   The ratio of correctly predicted positive observations to the total predicted positives.
*   Focuses on minimizing false positives.



In [48]:
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average='weighted')
precision

0.9293046576847748

### Recall

*   The ratio of correctly predicted positive observations to the actual positives.
*   Focuses on minimizing false negatives.



In [50]:
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred, average='weighted')
recall

0.9279907084785134

### F1Score

*   The harmonic mean of precision and recall, providing a balance between the two metrics.



In [52]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='weighted')
f1

0.9257487703287801

### Confusion Matrix

A table showing the true positive, true negative, false positive, and false negative values.

In [54]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[486,   0,   0,  29,   0,   0],
       [  2,   7,   0,   1,   0,   0],
       [  0,   0,   3,   0,   0,   0],
       [ 21,   0,   0, 296,   0,   0],
       [  5,   0,   0,   1,   2,   0],
       [  2,   0,   0,   1,   0,   5]])

### Classification Report

A summary of precision, recall, F1-score, and support for each class.

In [58]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       515
           1       1.00      0.70      0.82        10
           2       1.00      1.00      1.00         3
           3       0.90      0.93      0.92       317
           4       1.00      0.25      0.40         8
           5       1.00      0.62      0.77         8

    accuracy                           0.93       861
   macro avg       0.97      0.74      0.81       861
weighted avg       0.93      0.93      0.93       861



# Save and Reload the model

In [60]:
import joblib

# Save the model to disk
joblib.dump(best_model, 'model.joblib')

['model.joblib']

In [62]:
# Reload the model from disk
loaded_model = joblib.load('model.joblib')

# Extracting the heading and full article from unknown_articles.csv

In [63]:
import requests
from bs4 import BeautifulSoup

# Function to extract heading and full article content from a URL
def extract_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract heading
    heading = soup.find('h1').text.strip() if soup.find('h1') else None

    # Extract full article content
    article_content = '\n'.join([p.text.strip() for p in soup.find_all('p')])

    return heading, article_content

In [64]:
# Read URLs from the CSV file
df = pd.read_csv('unknown_articles.csv')

In [ ]:
# Create new columns for heading and full article
df['Heading'], df['Full_Article'] = zip(*df['Article.URL'].apply(extract_content))

In [ ]:
# Save the updated DataFrame to a new CSV file
df.to_csv('unknown_articles_extraction.csv', index=False)

# Predict the "Article_Types" for the extracted data

In [ ]:
def predict_article_types(df):

    # Vectorize the text columns using SentenceBERT
    for column in ['Heading', 'Full_Article']:
        df[column + '_embedding'] = df[column].apply(lambda x: model.encode(x))

    heading_features = pd.DataFrame(df['Heading_embedding'].to_list(), columns=[f'heading_{i}' for i in range(len(df['Heading_embedding'].iloc[0]))])
    article_features = pd.DataFrame(df['Full_Article_embedding'].to_list(), columns=[f'article_{i}' for i in range(len(df['Full_Article_embedding'].iloc[0]))])

    x = pd.concat([df, heading_features, article_features], axis=1)
    x = df1.drop(['Heading_embedding', 'Full_Article_embedding'], axis=1)

    # Scale the features
    x_scaled = scaler.transform(x)

    y_pred = loaded_model.predict(x_scaled)
    return y_pred


In [ ]:
predict_article_types(df)